In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt

plt.style.use("seaborn-white")
import seaborn as sns

sns.set_style("white")

from torch import nn

from tqdm import tqdm

In [2]:
from image_processing import downsample
from data import test_images_path, load_images_as_arrays, TGSSaltDataset, prepare_test_data
from visualisation import (
    plot_predictions,
)
from model import model_path, predict_tta
from metrics import iou_metric_batch
from toolz import compose
from data import rle_encode
import datetime
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch

from torch.utils import data

from resnetlike import UNetResNet
import logging
import random
import uuid
import os
from config import load_config

In [3]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [4]:
now = datetime.datetime.now()

In [5]:
config = load_config()['EvaluateModel']
logger.info(f'Loading config {config}')

INFO:config:Loading config ../configs/config.json
INFO:__main__:Loading config {'base_channels': 32, 'batch_size': 128, 'id': '61464e07-ce3e-42b1-b88a-482d645756d8', 'img_target_size': 101, 'initial_model_filename': 'best_lovasz_model.pth', 'name': 'EvaluateModel', 'num_workers': 0, 'seed': 42, 'threshold': 0.32277339226305085}


In [6]:
locals().update(config)

In [7]:
# img_size_target = 101
# batch_size = 128
# # learning_rate = 0.1
# # epochs = 70
# num_workers = 0
# seed = 42
# # num_cycles = (
# #     6
# # )  # Using Cosine Annealing with warm restarts, the number of times to oscillate
# # notebook_id = f"{now:%d%b%Y}_{uuid.uuid4()}"
# base_channels = 32
# # optim_config = {
# #     "optimizer": "sgd",
# #     "base_lr": 0.01,
# #     "momentum": 0.9,
# #     "weight_decay": 1e-4,
# #     "nesterov": True,
# #     "epochs": epochs,
# #     "scheduler": "cosine",
# #     "lr_min": 0,
# # }
# threshold_best=0.10676797542570594

In [8]:
torch.backends.cudnn.benchmark = True
logger.info(f"Started {now}")

INFO:__main__:Started 2018-10-05 12:42:03.903531


In [9]:
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

In [10]:
model = UNetResNet(1, base_channels)

In [11]:
device = torch.device("cuda:0")
model = nn.DataParallel(model)
model.to(device)

DataParallel(
  (module): UNetResNet(
    (enc1): EncodingLayer(
      (conv): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (res1): PreactivationResidualBlock(
        (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (res2): PreactivationResidualBlock(
        (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias

In [12]:
model.module.final_activation=nn.Sequential().to(device)

In [13]:
model_dir=os.path.join(model_path(), f"{id}")
filename = os.path.join(model_dir, initial_model_filename)
checkpoint = torch.load(filename)
model.load_state_dict(checkpoint["state_dict"])

In [14]:
test_df = prepare_test_data()
x_test = np.array(load_images_as_arrays(test_df.index, test_images_path()))
x_test = x_test.reshape(-1, 1, img_target_size, img_target_size)

100%|██████████| 18000/18000 [00:14<00:00, 1201.99it/s]


In [15]:
dataset_test = TGSSaltDataset(x_test, is_test=True)

In [16]:
test_data_loader = data.DataLoader(
    dataset_test,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True,
    drop_last=False,
)

In [17]:
model.eval()
predictions = [predict_tta(model, image) for image in tqdm(test_data_loader)]

100%|██████████| 141/141 [00:34<00:00,  4.78it/s]


In [18]:
preds_test = np.concatenate(predictions, axis=0).squeeze()

In [19]:
transform = compose(rle_encode, np.round, lambda x: x > threshold)

In [20]:
pred_dict = {
    idx: transform(preds_test[i]) for i, idx in enumerate(tqdm(test_df.index.values))
}

100%|██████████| 18000/18000 [00:05<00:00, 3204.81it/s]


In [21]:
sub = pd.DataFrame.from_dict(pred_dict, orient="index")
sub.index.names = ["id"]
sub.columns = ["rle_mask"]
filename = os.path.join(model_dir, f"submission_{now:%d%b%Y_%H}.csv")
sub.to_csv(filename)